<a href="https://colab.research.google.com/github/caio10012/Estudo_Python/blob/main/MLP/Backpropagation3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/valmirf/redes_neurais_pos.git

Cloning into 'redes_neurais_pos'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 120 (delta 11), reused 1 (delta 0), pack-reused 90 (from 1)
Receiving objects: 100% (120/120), 16.66 MiB | 10.35 MiB/s, done.
Resolving deltas: 100% (35/35), done.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation.

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [2]:
import random
import numpy as np

In [3]:
# Função de Ativação ReLU
def relu(net):
    return np.maximum(0, net)

# Função para retornar as derivadas da função ReLU
def relu_prime(z):
    return np.where(z > 0, 1.0, 0.0)


A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [4]:
# Classe Network modificada para usar ReLU
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)  # número de neurônios em cada camada
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]  # limiar
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]  # pesos

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = relu(np.dot(w, x) + b)  # usar ReLU em vez de sigmoid
        return x

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente."""
        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k + mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j, acc, n_test, (acc * 100) / n_test))
            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Atualiza os pesos e limiares da rede aplicando a descida do gradiente usando backpropagation para um único mini lote."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        self.weights = [w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o gradiente para a função de custo J_x."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        activation = x
        activations = [x]
        nets = []

        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation) + b
            nets.append(net)
            activation = relu(net)  # usar ReLU em vez de sigmoid
            activations.append(activation)

        delta = self.cost_derivative(activations[-1], y) * relu_prime(nets[-1])  # usar derivada de ReLU em vez de sigmoid_prime
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.num_layers):
            net = nets[-l]
            zs = relu_prime(net)  # usar derivada de ReLU em vez de sigmoid_prime
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * zs
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural produz o resultado correto."""
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations - y)


In [5]:
# Classe Network
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)  #número de neurônios em cada camada
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #limiar
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] #pesos

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = sigmoid(np.dot(w, x)+b) #net = (∑xw+b)
        return x

    def SGD(self, training_data, epochs, mini_batch_size, 𝜂, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente.
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""

        #dataset de treino
        training_data = list(training_data)
        n = len(training_data)

        #dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            #técnica que realiza o treinamento por lotes
            #mini_batch_size = tamanho do lote
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, 𝜂)

            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j,acc,n_test,(acc*100)/n_test));

            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, 𝜂):
        """Atualiza os pesos e limiares da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `a` é a taxa de aprendizado."""

        #inicializa matriz com derivadas de pesos e limiares
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
            #resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
            #soma os deltas do minibatch
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        #atualiza pesos e limiares (𝜂*𝛿*f’(net)*𝑥)
        self.weights = [w-(𝜂/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(𝜂/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo J_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        # Feedforward
        activation = x

        # Lista para armazenar todas as saídas dos neurônios (z), camada por camada
        activations = [x]

        # Lista para armazenar todos os vetores net, camada por camada
        nets = []

        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation)+b
            nets.append(net)
            activation = sigmoid(net) #z = valor de saída do neurônio
            activations.append(activation)

        # Backward pass

        #última camada -(u-z)f'(net)
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(nets[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #(𝑦−𝑧)*f’(net)*𝑥

        # l = 1 significa a última camada de neurônios, l = 2 é a penúltima e assim por diante.
        for l in range(2, self.num_layers):
            net = nets[-l]
            zs = sigmoid_prime(net)
            #delta da camada intermediaria. Note que utiliza o delta calculado anteriormente
            delta = np.dot(self.weights[-l+1].transpose(), delta) * zs
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) #∑(𝛿𝑤)f’(net)𝑥
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)

# Função de Ativação Sigmóide
def sigmoid(net):
    return 1.0/(1.0+np.exp(-net))

# Função para retornar as derivadas da função Sigmóide
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))


Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [9]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

# Carregar dados
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)


In [14]:


# Definir a arquitetura da rede com uma camada intermediária
arquitetura = [784, 200, 10]

# Taxa de aprendizado a ser testada
taxa_aprendizado = 0.3

# Função de ativação: Sigmoid
print("Testando função de ativação Sigmoid")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_sigmoid = Network(arquitetura)
rede_sigmoid.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

# Função de ativação: ReLU
print("Testando função de ativação ReLU")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_relu = Network(arquitetura)
rede_relu.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)


Testando função de ativação Sigmoid
Treinando com taxa de aprendizagem: 0.3
Epoch 0 : 2881 / 10000 = 28.81%
Epoch 1 : 3063 / 10000 = 30.63%
Epoch 2 : 3115 / 10000 = 31.15%
Epoch 3 : 3191 / 10000 = 31.91%
Epoch 4 : 3294 / 10000 = 32.94%
Epoch 5 : 3583 / 10000 = 35.83%
Epoch 6 : 3922 / 10000 = 39.22%
Epoch 7 : 4037 / 10000 = 40.37%
Epoch 8 : 4128 / 10000 = 41.28%
Epoch 9 : 4237 / 10000 = 42.37%
Testando função de ativação ReLU
Treinando com taxa de aprendizagem: 0.3
Epoch 0 : 1793 / 10000 = 17.93%
Epoch 1 : 1846 / 10000 = 18.46%
Epoch 2 : 1888 / 10000 = 18.88%
Epoch 3 : 1927 / 10000 = 19.27%
Epoch 4 : 1989 / 10000 = 19.89%
Epoch 5 : 2060 / 10000 = 20.6%
Epoch 6 : 2160 / 10000 = 21.6%
Epoch 7 : 2374 / 10000 = 23.74%
Epoch 8 : 2619 / 10000 = 26.19%
Epoch 9 : 2755 / 10000 = 27.55%


In [17]:
precisoes_sigmoide = [28.81, 30.63, 31.15, 31.91, 32.94, 35.83, 39.22, 40.37, 41.28, 42.37]
media_sigmoide = sum(precisoes_sigmoide) / len(precisoes_sigmoide)
print(f"Média de Precisão Sigmoide: {media_sigmoide:.2f}%")
precisoes_relu = [17.93, 18.46, 18.88, 19.27, 19.89, 20.60, 21.60, 23.74, 26.19, 27.55]
media_relu = sum(precisoes_relu) / len(precisoes_relu)
print(f"Média de Precisão ReLU: {media_relu:.2f}%")


Média de Precisão Sigmoide: 35.45%
Média de Precisão ReLU: 21.41%


In [18]:
# Definir a arquitetura da rede com uma camada intermediária
arquitetura = [784, 200, 10]

# Taxa de aprendizado a ser testada
taxa_aprendizado = 0.1

# Função de ativação: Sigmoid
print("Testando função de ativação Sigmoid")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_sigmoid = Network(arquitetura)
rede_sigmoid.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

# Função de ativação: ReLU
print("Testando função de ativação ReLU")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_relu = Network(arquitetura)
rede_relu.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

Testando função de ativação Sigmoid
Treinando com taxa de aprendizagem: 0.1
Epoch 0 : 3194 / 10000 = 31.94%
Epoch 1 : 3872 / 10000 = 38.72%
Epoch 2 : 4127 / 10000 = 41.27%
Epoch 3 : 4259 / 10000 = 42.59%
Epoch 4 : 4353 / 10000 = 43.53%
Epoch 5 : 4425 / 10000 = 44.25%
Epoch 6 : 4471 / 10000 = 44.71%
Epoch 7 : 4517 / 10000 = 45.17%
Epoch 8 : 4541 / 10000 = 45.41%
Epoch 9 : 4579 / 10000 = 45.79%
Testando função de ativação ReLU
Treinando com taxa de aprendizagem: 0.1
Epoch 0 : 3128 / 10000 = 31.28%
Epoch 1 : 3765 / 10000 = 37.65%
Epoch 2 : 4235 / 10000 = 42.35%
Epoch 3 : 4494 / 10000 = 44.94%
Epoch 4 : 4640 / 10000 = 46.4%
Epoch 5 : 4759 / 10000 = 47.59%
Epoch 6 : 4886 / 10000 = 48.86%
Epoch 7 : 5100 / 10000 = 51.0%
Epoch 8 : 5343 / 10000 = 53.43%
Epoch 9 : 5497 / 10000 = 54.97%


In [20]:
precisoes_sigmoide = [31.94, 38.72, 41.27, 42.59, 43.53, 44.25, 44.71, 45.17, 45.41, 45.79]
media_sigmoide = sum(precisoes_sigmoide) / len(precisoes_sigmoide)
print(f"Média de Precisão Sigmoide: {media_sigmoide:.2f}%")
precisoes_relu = [31.28, 37.65, 42.35, 44.94, 46.40, 47.59, 48.86, 51.00, 53.43, 54.97]
media_relu = sum(precisoes_relu) / len(precisoes_relu)
print(f"Média de Precisão ReLU: {media_relu:.2f}%")


Média de Precisão Sigmoide: 42.34%
Média de Precisão ReLU: 45.85%


In [19]:
# Definir a arquitetura da rede com uma camada intermediária
arquitetura = [784, 200, 10]

# Taxa de aprendizado a ser testada
taxa_aprendizado = 0.5

# Função de ativação: Sigmoid
print("Testando função de ativação Sigmoid")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_sigmoid = Network(arquitetura)
rede_sigmoid.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

# Função de ativação: ReLU
print("Testando função de ativação ReLU")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_relu = Network(arquitetura)
rede_relu.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

Testando função de ativação Sigmoid
Treinando com taxa de aprendizagem: 0.5
Epoch 0 : 4412 / 10000 = 44.12%
Epoch 1 : 5139 / 10000 = 51.39%
Epoch 2 : 5510 / 10000 = 55.1%
Epoch 3 : 5630 / 10000 = 56.3%
Epoch 4 : 6028 / 10000 = 60.28%
Epoch 5 : 6209 / 10000 = 62.09%
Epoch 6 : 6280 / 10000 = 62.8%
Epoch 7 : 6331 / 10000 = 63.31%
Epoch 8 : 6466 / 10000 = 64.66%
Epoch 9 : 7036 / 10000 = 70.36%
Testando função de ativação ReLU
Treinando com taxa de aprendizagem: 0.5
Epoch 0 : 1858 / 10000 = 18.58%
Epoch 1 : 2979 / 10000 = 29.79%
Epoch 2 : 4111 / 10000 = 41.11%
Epoch 3 : 4324 / 10000 = 43.24%
Epoch 4 : 4433 / 10000 = 44.33%
Epoch 5 : 4481 / 10000 = 44.81%
Epoch 6 : 4522 / 10000 = 45.22%
Epoch 7 : 5284 / 10000 = 52.84%
Epoch 8 : 5411 / 10000 = 54.11%
Epoch 9 : 5457 / 10000 = 54.57%


In [21]:
precisoes_sigmoide = [44.12, 51.39, 55.10, 56.30, 60.28, 62.09, 62.80, 63.31, 64.66, 70.36]
media_sigmoide = sum(precisoes_sigmoide) / len(precisoes_sigmoide)
print(f"Média de Precisão Sigmoide: {media_sigmoide:.2f}%")
precisoes_relu = [18.58, 29.79, 41.11, 43.24, 44.33, 44.81, 45.22, 52.84, 54.11, 54.57]
media_relu = sum(precisoes_relu) / len(precisoes_relu)
print(f"Média de Precisão ReLU: {media_relu:.2f}%")


Média de Precisão Sigmoide: 59.04%
Média de Precisão ReLU: 42.86%


In [22]:
# Definir a arquitetura da rede com uma camada intermediária
arquitetura = [784, 16, 10]

# Taxa de aprendizado a ser testada
taxa_aprendizado = 0.1

# Função de ativação: Sigmoid
print("Testando função de ativação Sigmoid")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_sigmoid = Network(arquitetura)
rede_sigmoid.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

# Função de ativação: ReLU
print("Testando função de ativação ReLU")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_relu = Network(arquitetura)
rede_relu.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

Testando função de ativação Sigmoid
Treinando com taxa de aprendizagem: 0.1
Epoch 0 : 2274 / 10000 = 22.74%
Epoch 1 : 3592 / 10000 = 35.92%
Epoch 2 : 4334 / 10000 = 43.34%
Epoch 3 : 4881 / 10000 = 48.81%
Epoch 4 : 5353 / 10000 = 53.53%
Epoch 5 : 5758 / 10000 = 57.58%
Epoch 6 : 6136 / 10000 = 61.36%
Epoch 7 : 6415 / 10000 = 64.15%
Epoch 8 : 6734 / 10000 = 67.34%
Epoch 9 : 7048 / 10000 = 70.48%
Testando função de ativação ReLU
Treinando com taxa de aprendizagem: 0.1
Epoch 0 : 2608 / 10000 = 26.08%
Epoch 1 : 3833 / 10000 = 38.33%
Epoch 2 : 4782 / 10000 = 47.82%
Epoch 3 : 5348 / 10000 = 53.48%
Epoch 4 : 5664 / 10000 = 56.64%
Epoch 5 : 5911 / 10000 = 59.11%
Epoch 6 : 6148 / 10000 = 61.48%
Epoch 7 : 6363 / 10000 = 63.63%
Epoch 8 : 6640 / 10000 = 66.4%
Epoch 9 : 6871 / 10000 = 68.71%


In [25]:
precisoes_sigmoide = [22.74, 35.92, 43.34, 48.81, 53.53, 57.58, 61.36, 64.15, 67.34, 70.48]
media_sigmoide = sum(precisoes_sigmoide) / len(precisoes_sigmoide)
print(f"Média de Precisão Sigmoide: {media_sigmoide:.2f}%")
precisoes_relu = [26.08, 38.33, 47.82, 53.48, 56.64, 59.11, 61.48, 63.63, 66.40, 68.71]
media_relu = sum(precisoes_relu) / len(precisoes_relu)
print(f"Média de Precisão ReLU: {media_relu:.2f}%")


Média de Precisão Sigmoide: 52.53%
Média de Precisão ReLU: 54.17%


In [23]:
# Definir a arquitetura da rede com uma camada intermediária
arquitetura = [784, 16, 10]

# Taxa de aprendizado a ser testada
taxa_aprendizado = 0.3

# Função de ativação: Sigmoid
print("Testando função de ativação Sigmoid")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_sigmoid = Network(arquitetura)
rede_sigmoid.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

# Função de ativação: ReLU
print("Testando função de ativação ReLU")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_relu = Network(arquitetura)
rede_relu.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

Testando função de ativação Sigmoid
Treinando com taxa de aprendizagem: 0.3
Epoch 0 : 4440 / 10000 = 44.4%
Epoch 1 : 5765 / 10000 = 57.65%
Epoch 2 : 6449 / 10000 = 64.49%
Epoch 3 : 7520 / 10000 = 75.2%
Epoch 4 : 7980 / 10000 = 79.8%
Epoch 5 : 8182 / 10000 = 81.82%
Epoch 6 : 8340 / 10000 = 83.4%
Epoch 7 : 8446 / 10000 = 84.46%
Epoch 8 : 8556 / 10000 = 85.56%
Epoch 9 : 8625 / 10000 = 86.25%
Testando função de ativação ReLU
Treinando com taxa de aprendizagem: 0.3
Epoch 0 : 4781 / 10000 = 47.81%
Epoch 1 : 6262 / 10000 = 62.62%
Epoch 2 : 6606 / 10000 = 66.06%
Epoch 3 : 7250 / 10000 = 72.5%
Epoch 4 : 7554 / 10000 = 75.54%
Epoch 5 : 7753 / 10000 = 77.53%
Epoch 6 : 7862 / 10000 = 78.62%
Epoch 7 : 7953 / 10000 = 79.53%
Epoch 8 : 8017 / 10000 = 80.17%
Epoch 9 : 8148 / 10000 = 81.48%


In [26]:
# Precisões Sigmoide
precisoes_sigmoide = [44.40, 57.65, 64.49, 75.20, 79.80, 81.82, 83.40, 84.46, 85.56, 86.25]
media_sigmoide = sum(precisoes_sigmoide) / len(precisoes_sigmoide)
print(f"Média de Precisão Sigmoide: {media_sigmoide:.2f}%")

# Precisões ReLU
precisoes_relu = [47.81, 62.62, 66.06, 72.50, 75.54, 77.53, 78.62, 79.53, 80.17, 81.48]
media_relu = sum(precisoes_relu) / len(precisoes_relu)
print(f"Média de Precisão ReLU: {media_relu:.2f}%")


Média de Precisão Sigmoide: 74.30%
Média de Precisão ReLU: 72.19%


In [24]:
# Definir a arquitetura da rede com uma camada intermediária
arquitetura = [784, 16, 10]

# Taxa de aprendizado a ser testada
taxa_aprendizado = 0.5

# Função de ativação: Sigmoid
print("Testando função de ativação Sigmoid")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_sigmoid = Network(arquitetura)
rede_sigmoid.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

# Função de ativação: ReLU
print("Testando função de ativação ReLU")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_relu = Network(arquitetura)
rede_relu.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

Testando função de ativação Sigmoid
Treinando com taxa de aprendizagem: 0.5
Epoch 0 : 5046 / 10000 = 50.46%
Epoch 1 : 7424 / 10000 = 74.24%
Epoch 2 : 8055 / 10000 = 80.55%
Epoch 3 : 8358 / 10000 = 83.58%
Epoch 4 : 8509 / 10000 = 85.09%
Epoch 5 : 8616 / 10000 = 86.16%
Epoch 6 : 8686 / 10000 = 86.86%
Epoch 7 : 8745 / 10000 = 87.45%
Epoch 8 : 8786 / 10000 = 87.86%
Epoch 9 : 8824 / 10000 = 88.24%
Testando função de ativação ReLU
Treinando com taxa de aprendizagem: 0.5
Epoch 0 : 4696 / 10000 = 46.96%
Epoch 1 : 6572 / 10000 = 65.72%
Epoch 2 : 7162 / 10000 = 71.62%
Epoch 3 : 7446 / 10000 = 74.46%
Epoch 4 : 7637 / 10000 = 76.37%
Epoch 5 : 7748 / 10000 = 77.48%
Epoch 6 : 7842 / 10000 = 78.42%
Epoch 7 : 7886 / 10000 = 78.86%
Epoch 8 : 7928 / 10000 = 79.28%
Epoch 9 : 7961 / 10000 = 79.61%


In [27]:
# Precisões Sigmoide
precisoes_sigmoide = [50.46, 74.24, 80.55, 83.58, 85.09, 86.16, 86.86, 87.45, 87.86, 88.24]
media_sigmoide = sum(precisoes_sigmoide) / len(precisoes_sigmoide)
print(f"Média de Precisão Sigmoide: {media_sigmoide:.2f}%")

# Precisões ReLU
precisoes_relu = [46.96, 65.72, 71.62, 74.46, 76.37, 77.48, 78.42, 78.86, 79.28, 79.61]
media_relu = sum(precisoes_relu) / len(precisoes_relu)
print(f"Média de Precisão ReLU: {media_relu:.2f}%")


Média de Precisão Sigmoide: 81.05%
Média de Precisão ReLU: 72.88%


In [28]:
# Definir a arquitetura da rede com uma camada intermediária
arquitetura = [784, 32, 10]

# Taxa de aprendizado a ser testada
taxa_aprendizado = 0.1

# Função de ativação: Sigmoid
print("Testando função de ativação Sigmoid")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_sigmoid = Network(arquitetura)
rede_sigmoid.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

# Função de ativação: ReLU
print("Testando função de ativação ReLU")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_relu = Network(arquitetura)
rede_relu.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

Testando função de ativação Sigmoid
Treinando com taxa de aprendizagem: 0.1
Epoch 0 : 2971 / 10000 = 29.71%
Epoch 1 : 4263 / 10000 = 42.63%
Epoch 2 : 5036 / 10000 = 50.36%
Epoch 3 : 5700 / 10000 = 57.0%
Epoch 4 : 6102 / 10000 = 61.02%
Epoch 5 : 6372 / 10000 = 63.72%
Epoch 6 : 6563 / 10000 = 65.63%
Epoch 7 : 6778 / 10000 = 67.78%
Epoch 8 : 7044 / 10000 = 70.44%
Epoch 9 : 7268 / 10000 = 72.68%
Testando função de ativação ReLU
Treinando com taxa de aprendizagem: 0.1
Epoch 0 : 1997 / 10000 = 19.97%
Epoch 1 : 2798 / 10000 = 27.98%
Epoch 2 : 3800 / 10000 = 38.0%
Epoch 3 : 4867 / 10000 = 48.67%
Epoch 4 : 5755 / 10000 = 57.55%
Epoch 5 : 6229 / 10000 = 62.29%
Epoch 6 : 6514 / 10000 = 65.14%
Epoch 7 : 6723 / 10000 = 67.23%
Epoch 8 : 6896 / 10000 = 68.96%
Epoch 9 : 7056 / 10000 = 70.56%


In [31]:
# Precisões Sigmoide
precisoes_sigmoide = [29.71, 42.63, 50.36, 57.00, 61.02, 63.72, 65.63, 67.78, 70.44, 72.68]
media_sigmoide = sum(precisoes_sigmoide) / len(precisoes_sigmoide)
print(f"Média de Precisão Sigmoide: {media_sigmoide:.2f}%")

# Precisões ReLU
precisoes_relu = [19.97, 27.98, 38.00, 48.67, 57.55, 62.29, 65.14, 67.23, 68.96, 70.56]
media_relu = sum(precisoes_relu) / len(precisoes_relu)
print(f"Média de Precisão ReLU: {media_relu:.2f}%")


Média de Precisão Sigmoide: 58.10%
Média de Precisão ReLU: 52.64%


In [29]:
# Definir a arquitetura da rede com uma camada intermediária
arquitetura = [784, 32, 10]

# Taxa de aprendizado a ser testada
taxa_aprendizado = 0.3

# Função de ativação: Sigmoid
print("Testando função de ativação Sigmoid")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_sigmoid = Network(arquitetura)
rede_sigmoid.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

# Função de ativação: ReLU
print("Testando função de ativação ReLU")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_relu = Network(arquitetura)
rede_relu.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

Testando função de ativação Sigmoid
Treinando com taxa de aprendizagem: 0.3
Epoch 0 : 4862 / 10000 = 48.62%
Epoch 1 : 6712 / 10000 = 67.12%
Epoch 2 : 7289 / 10000 = 72.89%
Epoch 3 : 7608 / 10000 = 76.08%
Epoch 4 : 8199 / 10000 = 81.99%
Epoch 5 : 8453 / 10000 = 84.53%
Epoch 6 : 8558 / 10000 = 85.58%
Epoch 7 : 8650 / 10000 = 86.5%
Epoch 8 : 8717 / 10000 = 87.17%
Epoch 9 : 8755 / 10000 = 87.55%
Testando função de ativação ReLU
Treinando com taxa de aprendizagem: 0.3
Epoch 0 : 5095 / 10000 = 50.95%
Epoch 1 : 6254 / 10000 = 62.54%
Epoch 2 : 7379 / 10000 = 73.79%
Epoch 3 : 7999 / 10000 = 79.99%
Epoch 4 : 8262 / 10000 = 82.62%
Epoch 5 : 8444 / 10000 = 84.44%
Epoch 6 : 8561 / 10000 = 85.61%
Epoch 7 : 8653 / 10000 = 86.53%
Epoch 8 : 8722 / 10000 = 87.22%
Epoch 9 : 8781 / 10000 = 87.81%


In [32]:
# Precisões Sigmoide
precisoes_sigmoide = [48.62, 67.12, 72.89, 76.08, 81.99, 84.53, 85.58, 86.50, 87.17, 87.55]
media_sigmoide = sum(precisoes_sigmoide) / len(precisoes_sigmoide)
print(f"Média de Precisão Sigmoide: {media_sigmoide:.2f}%")

# Precisões ReLU
precisoes_relu = [50.95, 62.54, 73.79, 79.99, 82.62, 84.44, 85.61, 86.53, 87.22, 87.81]
media_relu = sum(precisoes_relu) / len(precisoes_relu)
print(f"Média de Precisão ReLU: {media_relu:.2f}%")


Média de Precisão Sigmoide: 77.80%
Média de Precisão ReLU: 78.15%


In [30]:
# Definir a arquitetura da rede com uma camada intermediária
arquitetura = [784, 32, 10]

# Taxa de aprendizado a ser testada
taxa_aprendizado = 0.5

# Função de ativação: Sigmoid
print("Testando função de ativação Sigmoid")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_sigmoid = Network(arquitetura)
rede_sigmoid.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

# Função de ativação: ReLU
print("Testando função de ativação ReLU")

print(f"Treinando com taxa de aprendizagem: {taxa_aprendizado}")
rede_relu = Network(arquitetura)
rede_relu.SGD(training_data, 10, 32, taxa_aprendizado, test_data=test_data_list)

Testando função de ativação Sigmoid
Treinando com taxa de aprendizagem: 0.5
Epoch 0 : 5962 / 10000 = 59.62%
Epoch 1 : 7579 / 10000 = 75.79%
Epoch 2 : 8010 / 10000 = 80.1%
Epoch 3 : 8424 / 10000 = 84.24%
Epoch 4 : 8635 / 10000 = 86.35%
Epoch 5 : 8746 / 10000 = 87.46%
Epoch 6 : 8816 / 10000 = 88.16%
Epoch 7 : 8874 / 10000 = 88.74%
Epoch 8 : 8899 / 10000 = 88.99%
Epoch 9 : 8957 / 10000 = 89.57%
Testando função de ativação ReLU
Treinando com taxa de aprendizagem: 0.5
Epoch 0 : 5378 / 10000 = 53.78%
Epoch 1 : 6661 / 10000 = 66.61%
Epoch 2 : 7198 / 10000 = 71.98%
Epoch 3 : 7455 / 10000 = 74.55%
Epoch 4 : 7607 / 10000 = 76.07%
Epoch 5 : 8614 / 10000 = 86.14%
Epoch 6 : 8795 / 10000 = 87.95%
Epoch 7 : 8857 / 10000 = 88.57%
Epoch 8 : 8910 / 10000 = 89.1%
Epoch 9 : 8951 / 10000 = 89.51%


In [33]:
# Precisões Sigmoide
precisoes_sigmoide = [59.62, 75.79, 80.10, 84.24, 86.35, 87.46, 88.16, 88.74, 88.99, 89.57]
media_sigmoide = sum(precisoes_sigmoide) / len(precisoes_sigmoide)
print(f"Média de Precisão Sigmoide: {media_sigmoide:.2f}%")

# Precisões ReLU
precisoes_relu = [53.78, 66.61, 71.98, 74.55, 76.07, 86.14, 87.95, 88.57, 89.10, 89.51]
media_relu = sum(precisoes_relu) / len(precisoes_relu)
print(f"Média de Precisão ReLU: {media_relu:.2f}%")


Média de Precisão Sigmoide: 82.90%
Média de Precisão ReLU: 78.43%


#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




In [8]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

#arquitetura da rede
arquitecture = [784, 30, 20, 10]
mlp = Network(arquitecture)
mlp.SGD(training_data, 10, 32, 0.3, test_data=test_data)


Epoch 0 : 4984 / 10000 = 49.84%


KeyboardInterrupt: 

##**Mini-Projeto**
1) Realizar avaliações modificando os seguintes parâmetros:     

     a) Taxa de aprendizagem: 0.1, 0.3 e 0.5
     b) Função de ativação RELU
     c) Rede com uma camada intermediária com 3 configurações diferentes (Explicite a configuração utilizada)
     d) Rede com duas camadas intermediárias com 3 configurações diferentes (Explicite a configuração utilizada)
     
      
Complete a Tabela abaixo com os resultados (Pra cada configuração de camadas intermediárias, execute as 3 taxas de aprendizagem pra função de ativação Sigmoide e Relu):


\begin{array}{|c|ccc|ccc|ccc|}\hline\\ \\
  1\;Camada\;Intermediária & & \mathcal{𝜂=0.1} & &  & \mathcal{𝜂=0.3} & & & \mathcal{𝜂=0.5} &  & \\ \hline
Configurações & [784, 16, 10], [784, 32, 10], [784, 64, 10] &&&[784, 16, 10], [784, 32, 10], [784, 64, 10] &&&[784, 16, 10], [784, 32, 10], [784, 64, 10]\\ \hline
Sigmoide  & & & &a &  & & &  &  & \\ \hline
Relu  & & & &a  &  & & &  &  & \\ \hline
  2\;Camadas\;Intermediárias & & \mathcal{𝜂=0.1} & &  & \mathcal{𝜂=0.3} & & & \mathcal{𝜂=0.5} &  & \\ \hline
Configurações & 1 & 2 & 3 & 1 & 2 & 3 & 1 & 2 & 3 & \\ \hline
Sigmoide  & & & &  &  & & &  &  & \\ \hline
Relu  & & & &  &  & & &  &  & \\ \hline
\end{array}

2) Modifique a taxa de aprendizagem pra diminuir com o tempo. Execute com a melhor configuração encontrada. Melhorou o resultado?
